Use of Dynamic Time Warping (DTW Algorithm) to compare two time series data, given that the movement of 2 stock price data do not occur simultaneously. 

Sources
- dtwParallel: A Python package to efficiently compute dynamic time warping between time series
https://www.sciencedirect.com/science/article/pii/S2352711023000602

- explanation : https://www.theaidream.com/post/dynamic-time-warping-dtw-algorithm-in-time-series

Explanation

The "corr_twoVar" utilizes the DTW Algorithm to identify similarity levels of a given metric


Alternatives
- If DTW is time-inefficient, FAST DTW (or) Sparce DTW can be used
- If 'similarity' is not sufficient, each stock being compared can be shifted by different time intervals, and then compared with standard PPMCC Corellations

In [1]:
import pandas as pd
import numpy as np

# Plotting Packages
import matplotlib.pyplot as plt
import seaborn as sbn

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150
savefig_options = dict(format="png", dpi=150, bbox_inches="tight")

# Computation packages
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

In [ ]:
"""Calculate distance matrix
This method calcualtes the pairwise Euclidean distance between two sequences.
The sequences can have different lengths.
"""

def compute_euclidean_distance_matrix(x, y) -> np.array:
    dist = np.zeros((len(y), len(x)))
    for i in range(len(y)):
        for j in range(len(x)):
            dist[i,j] = (x[j]-y[i])**2
    return dist

In [ ]:
"""Compute accumulated cost matrix for warp path using Euclidean distance
"""

def compute_accumulated_cost_matrix(x, y) -> np.array:
    distances = compute_euclidean_distance_matrix(x, y)

    # Initialization
    cost = np.zeros((len(y), len(x)))
    cost[0,0] = distances[0,0]
    
    for i in range(1, len(y)):
        cost[i, 0] = distances[i, 0] + cost[i-1, 0]  
        
    for j in range(1, len(x)):
        cost[0, j] = distances[0, j] + cost[0, j-1]  

    # Accumulated warp path cost
    for i in range(1, len(y)):
        for j in range(1, len(x)):
            cost[i, j] = min(
                cost[i-1, j],    # insertion
                cost[i, j-1],    # deletion
                cost[i-1, j-1]   # match
            ) + distances[i, j] 
            
    return cost